In [147]:
import pandas as pd
import numpy as np
import sqlite3
import importlib
from datetime import datetime

# keeping company information in additional file
import data_file

In [ ]:
importlib.reload(data_file)

In [148]:
sales_person_n = data_file.sales_person_n
exlbu=data_file.exlbu
exlpanrter=data_file.exlpanrter

# Get current year and month
now = datetime.now()
last_month = now
# Format last month as string in YYYYMM format
year_month = last_month.strftime("%Y%m")

In [149]:
conn = sqlite3.connect('data_files/order_data.db')
query = "SELECT * FROM orders"
df_orders = pd.read_sql_query(query, conn)
conn.close()

conn2 = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers"
df_customers = pd.read_sql_query(query, conn2)
conn2.close()

In [150]:
# for testing purposes I keep this lines
wdf = df_customers.copy()
dfc = df_orders.copy()

In [151]:
wdf.loc[wdf['tier'].isna(), 'tier'] = 'Direct'
wdf.loc[wdf['tier'] == 'None', 'tier'] = 'Direct'
wdf.loc[wdf['tier_new'].isna(), 'tier_new'] = 'Direct'
wdf.loc[wdf['tier_new'] == 'None', 'tier_new'] = 'Direct'

In [152]:
bu_defin = pd.read_excel('data_files/bu_defin.xlsx')
bu_defin = bu_defin.loc[:, 'bu2':]
#bu_defin = df_orders.loc[:, ['bu2', 'for_bu']]
#bu_defin.dropna(inplace=True)
#bu_defin.drop_duplicates(inplace=True)
#bu_defin.reset_index(inplace=True, drop=True)
#bu_defin.loc[bu_defin['bu2'] == exlbu, 'for_bu'] = 'PCI Transmitters'
bu_defin['for_bu'] = bu_defin['for_bu'].astype(str)
bu_defin['bu2'] = bu_defin['bu2'].astype(str)
bu_defin['bu2'] = bu_defin['bu2'].str.strip()

In [153]:
order_data = dfc.loc[:,['company_code_n', 'year_month', 'FY', 
       'bu2', 'sales_person_n', 'sales_order_so', 'sold_to_customer',
       'sold_to_customer_n','order_intake_amount_eur', 'ec_eu_customer_n']]

order_data['bu2'] = order_data['bu2'].astype(str)
order_data['bu2'] = order_data['bu2'].str.strip()

In [154]:
# add bu column for future reports
order_data = order_data.merge(bu_defin, how='left' )

In [155]:
#order_data = order_data.drop('Unnamed: 0', axis=1)
#order_data = order_data.drop('for_bu', axis=1)

In [156]:
order_columns = list(order_data.columns) + ['customer_name', 'indirect_direct', 'tier', 'tier_new', 'type', 'countries']

In [ ]:
# special case with specific SO for agent
agent_so = data_file.special_so
agent_so_two = data_file.special_so_two 
agent_one = data_file.agent_one


special_so_three = data_file.special_so_three
agent_three = data_file.agent_three

so_order_data = order_data[ (order_data['sales_order_so'] == agent_so) | (order_data['sales_order_so'] == agent_so_two) | (order_data['sales_order_so'] == special_so_three)]

so_order_data.loc[so_order_data['sales_order_so'] != special_so_three, 'customer_name'] = agent_one
so_order_data.loc[so_order_data['sales_order_so'] == special_so_three, 'customer_name'] = agent_three

so_order_data['indirect_direct'] = 'Indirect'
so_order_data['tier'] = 'Channel Partner'
so_order_data['tier_new'] = 'Channel Partner'
so_order_data['type'] = 'Agent'

In [158]:
# special case with specific cutomers for agent
partners_data = wdf[(wdf['tier'] != 'Direct') | wdf[exlpanrter].notna()]
agent_two = data_file.agent_two
partners_data.loc[partners_data[exlpanrter].notna(), 'customer_name'] = agent_two
partners_data = partners_data.loc[:, ['sold_to_customer', 'agent_person', 'company_code_n', 'sold_to_customer_n','customer_name', 
       'indirect_direct', 'channel', 'type', 'tier', 'tier_new', 'countries']]

partners_data['sold_to_customer'] = partners_data['sold_to_customer'].astype(str)

In [159]:
agent_partners = partners_data[partners_data['agent_person'].notna()]
agent_partners = agent_partners[agent_partners['agent_person'] != 'None']
# agent sales records preparation
agents_order_data = order_data[order_data['sales_person_n'].isin(sales_person_n)]
agents_order_data = agents_order_data.merge(agent_partners, left_on='sales_person_n', right_on='sold_to_customer_n', how='left')
agents_order_data = agents_order_data.filter(regex='^(?!.*_y)')
# Exclude '_x' from all column names
agents_order_data.columns = agents_order_data.columns.str.replace('_x', '')
agents_order_data = agents_order_data.loc[:, order_columns]
agents_order_data['type'] = 'Agent'

In [160]:
# exclude agents sales from order data not to double lines
order_data = order_data[~order_data['sales_person_n'].isin(sales_person_n)]
# all partners except agents data preparation
other_order_data = order_data[order_data['sold_to_customer'].isin(partners_data['sold_to_customer'])]
other_order_data = other_order_data.merge(partners_data, left_on='sold_to_customer', right_on='sold_to_customer', how='left')
other_order_data = other_order_data.filter(regex='^(?!.*_y)')
# Exclude '_x' from all column names
other_order_data.columns = other_order_data.columns.str.replace('_x', '')
other_order_data = other_order_data.loc[:, order_columns]
# Update values in columns based on the specific value
other_order_data.loc[other_order_data['indirect_direct'] == 'Direct', 'tier'] = 'Channel Partner'
other_order_data.loc[other_order_data['indirect_direct'] == 'Direct', 'tier_new'] = 'Channel Partner'
other_order_data.loc[other_order_data['indirect_direct'] == 'Direct', 'type'] = 'Agent'

In [161]:
# combining all the sales records
full_data = pd.concat([other_order_data, agents_order_data, so_order_data])
full_data.reset_index(inplace=True, drop=True)
#full_data.drop_duplicates(inplace=True)

In [162]:
# for testing purposes I keep this lines
full_data_df = full_data.copy()
print(len(full_data))

In [164]:
#clean other division products

# define a list of products which I want to mark separetely
product_list = data_file.tm_product_list
tm_threshold = 0.9

# define total sum of orders for each customer
total_sum = full_data_df.groupby('customer_name')['order_intake_amount_eur'].sum()
total_sum = total_sum.rename('total_OI')
# Merge with the original DataFrame to include all products and companies
full_data_df = pd.merge(full_data_df, total_sum, on='customer_name', how='left')
# Fill NaN values in 'sum' column with '0'
full_data_df['total_OI'] = full_data_df['total_OI'].fillna(0)

# Calculate the sum of sales for the products which I want to mark across all companies
total_tm = full_data_df[full_data_df['bu2'].isin(product_list)].groupby('customer_name')['order_intake_amount_eur'].sum().reset_index()
# Rename the 'order_intake_amount_eur' column to 'tm_sum'
total_tm.rename(columns={'order_intake_amount_eur': 'tm_sum'}, inplace=True)
# Merge with the original DataFrame to include all products and companies
full_data_df = pd.merge(full_data_df, total_tm, on='customer_name', how='left')
# Fill NaN values in 'sum' column with '0'
full_data_df['tm_sum'] = full_data_df['tm_sum'].fillna(0)

# calculate the percentage of sales for the products in the list for each company
full_data_df['tm_share'] = full_data_df['tm_sum'] / full_data_df['total_OI']
full_data_df['tm_share'] = full_data_df['tm_share'].fillna(0)

# label companies where the percentage of sales for all products in the list is greater than treshold value
full_data_df['tm_check'] = 'no'
full_data_df.loc[full_data_df['tm_share'] > tm_threshold, 'tm_check'] = 'yes'

In [165]:
# marking special case

special_one = data_file.special_one
special_one_office = data_file.special_one_office
# Filter the DataFrame based on the conditions
full_data_df.loc[(full_data_df['customer_name'] == special_one) & (full_data_df['company_code_n'] == special_one_office) , 'tm_check'] = 'no'
full_data_df.loc[(full_data_df['customer_name'] == special_one) & (full_data_df['company_code_n'] != special_one_office) , 'tier'] = 'Independent Sales Company'
full_data_df.loc[(full_data_df['customer_name'] == special_one) & (full_data_df['company_code_n'] != special_one_office) , 'tier_new'] = 'Independent Sales Company'
full_data_df.loc[(full_data_df['customer_name'] == special_one) & (full_data_df['company_code_n'] != special_one_office) , 'type'] = 'Trading Firm'

special_two = data_file.special_two
special_two_file_name = data_file.special_two_file_name

# Filter the DataFrame based on the conditions
# Convert 'data' column to datetime format
full_data_df['FY'] = pd.to_datetime(full_data_df['FY'])

special_two_so = pd.read_excel(special_two_file_name)
condition_a = full_data_df['sales_person_n'] != special_two
condition_b = full_data_df['sales_person_n'] == special_two
condition_c = full_data_df['sales_order_so'].isin(special_two_so['sales_order_so'])
condition_d = full_data_df['FY'].dt.year > 2022
full_data_df = full_data_df.loc[ condition_a | (condition_b & condition_c) | (condition_b & condition_d)]

In [166]:
tm_exclusion_office = data_file.tm_exclusion_office
special_three = data_file.special_three
special_four = data_file.special_four
special_five = data_file.special_five
special_six = data_file.special_six
special_six_office = data_file.special_six_office
special_seven = data_file.special_seven


# eclusion case for t&m 
exl_condition_a = (full_data_df['FY'].dt.year == 2022) | (full_data_df['FY'].dt.year == 2021)
exl_condition_at = full_data_df['FY'].dt.year < 2023
exl_condition_b = full_data_df['company_code_n'] != tm_exclusion_office
exl_condition_c = full_data_df['tm_check'] == 'yes'
exl_condition_d = full_data_df['sold_to_customer'] == special_three
exl_condition_e = full_data_df['customer_name'] == special_four
exl_condition_f = full_data_df['customer_name'] == special_five

exl_condition_j = full_data_df['customer_name'] == special_six
exl_condition_i = full_data_df['company_code_n'].isin(special_six_office)

exl_condition_k = full_data_df['customer_name'].isin(special_seven)

#full_data_df.loc[exl_condition_at & exl_condition_k, 'tm_check'] = 'new'
#full_data_df.loc[exl_condition_at & exl_condition_d, 'tm_check'] = 'new'
#full_data_df.loc[exl_condition_j & exl_condition_i & exl_condition_at, 'tm_check'] = 'new'

full_data_df.loc[ (exl_condition_a & exl_condition_e) | (exl_condition_a & exl_condition_f) | (exl_condition_a & exl_condition_b & exl_condition_c), 'tm_check'] = 'no'

In [167]:
special_eight = data_file.special_eight[0]
exl_condition_special_eight_name = full_data_df['customer_name'] == special_eight
exl_condition_special_eight = full_data_df['FY'].dt.year > 2022

full_data_df.loc[ exl_condition_special_eight_name & exl_condition_special_eight , 'tier_new'] = 'Indirect Business Partner'
print(len(full_data_df))

In [169]:
# Define the desired column order
column_order = ['sold_to_customer','customer_name', 'sold_to_customer_n', 'ec_eu_customer_n' ,'company_code_n', 'countries', 'indirect_direct', 'tier', 'tier_new', 'type', 'year_month', 'FY', 
       'bu2', 'for_bu', 'sales_order_so', 'sales_person_n','order_intake_amount_eur', 'tm_check', 'tm_share']

# Reorder the columns
full_data_df = full_data_df[column_order]

In [170]:
# other way to indentify T&M customers

#clean other division products

# define a list of products which I want to mark separetely
product_list = data_file.tm_product_list
pers_tm_threshold = 0.5

# define total sum of orders for each customer
person_total_sum = full_data_df.groupby('sales_person_n')['order_intake_amount_eur'].sum()
person_total_sum = person_total_sum.rename('person_total_OI')
# Merge with the original DataFrame to include all products and companies
full_data_df = pd.merge(full_data_df, person_total_sum, on='sales_person_n', how='left')
# Fill NaN values in 'sum' column with '0'
full_data_df['person_total_OI'] = full_data_df['person_total_OI'].fillna(0)

# Calculate the sum of sales for the products which I want to mark across all companies
person_total_tm = full_data_df[full_data_df['bu2'].isin(product_list)].groupby('sales_person_n')['order_intake_amount_eur'].sum().reset_index()
# Rename the 'order_intake_amount_eur' column to 'tm_sum'
person_total_tm.rename(columns={'order_intake_amount_eur': 'pers_tm_sum'}, inplace=True)
# Merge with the original DataFrame to include all products and companies
full_data_df = pd.merge(full_data_df, person_total_tm, on='sales_person_n', how='left')
# Fill NaN values in 'sum' column with '0'
full_data_df['pers_tm_sum'] = full_data_df['pers_tm_sum'].fillna(0)

# calculate the percentage of sales for the products in the list for each company
full_data_df['pers_tm_share'] = full_data_df['pers_tm_sum'] / full_data_df['person_total_OI']
full_data_df['pers_tm_share'] = full_data_df['pers_tm_share'].fillna(0)

# label companies where the percentage of sales for all products in the list is greater than treshold value
full_data_df['pers_tm_check'] = 'no'
full_data_df.loc[full_data_df['pers_tm_share'] > pers_tm_threshold, 'pers_tm_check'] = 'yes'

In [171]:
# Filter companies with only 'yes' values in column P
filtered_companies = full_data_df[full_data_df['pers_tm_check'] == 'yes']['sold_to_customer'].tolist()
filtered_companies_no = full_data_df[full_data_df['pers_tm_check'] == 'no']['sold_to_customer'].tolist()
filtered_companies = [x for x in filtered_companies if x not in filtered_companies_no]
filtered_companies = list(set(filtered_companies))

full_data_df['pure tm'] = 'no'
full_data_df.loc[full_data_df['sold_to_customer'].isin(filtered_companies), 'pure tm'] = 'yes'

In [172]:
special_nine = data_file.special_nine
add_condition_a = full_data_df['sold_to_customer'] == special_nine

full_data_df.loc[add_condition_a, 'tm_check'] = 'no'
full_data_df.loc[add_condition_a, 'pure tm'] = 'no'

In [173]:
condition_one = full_data_df['tier'] == 'Independent Sales Company'
condition_two = full_data_df['FY'].dt.year == 2022

full_data_df.loc[condition_one & condition_two, 'pure tm'] = 'no'

In [ ]:
"""
writer = pd.ExcelWriter(f'data_files/outcome/results_pers_{year_month}.xlsx')
# Save each DataFrame to a separate sheet in the same file
full_data_df.to_excel(writer, sheet_name='results', index=False)
# Save the file
writer.close()
"""

In [174]:
# add budget information
targets = pd.read_excel('data_files/outcome/target.xlsx')
targets['order_intake_amount_eur'] = 0
# Convert the 'FY' column to datetime format
targets['FY'] = pd.to_datetime(targets['FY'], format='%d-%m-%Y')


temp = full_data_df.copy()
temp['target_order_intake_amount_eur'] = 0

temp = pd.concat([temp, targets], ignore_index=True)
temp['m_num'] = temp['FY'].dt.month

In [175]:
# addition of target numbers for two business tiers

condition_isc = temp['tier_new'] == 'Independent Sales Company'
condition_ibp = temp['tier_new'] == 'Indirect Business Partner'

condition_tm = (temp['tm_check'] == 'no') & (temp['pure tm'] == 'no')

result_isc = temp[(temp['FY'].dt.year == 2022) & condition_isc & condition_tm]['order_intake_amount_eur'].sum()
target_isc = temp[(temp['FY'].dt.year == 2023) & condition_isc & condition_tm]['target_order_intake_amount_eur'].sum()

target_isc_should_be_by_month = (round(result_isc * 1.1, -3) - target_isc) / 12

result_ibp = temp[(temp['FY'].dt.year == 2022) & condition_ibp & condition_tm]['order_intake_amount_eur'].sum()
target_ibp = temp[(temp['FY'].dt.year == 2023) & condition_ibp & condition_tm]['target_order_intake_amount_eur'].sum()

target_ibp_should_be_by_month = (round(result_ibp * 1.1, -3) - target_ibp) / 12

fy_23 =  [202304, 202305, 202306, 202307, 202308, 202309, 202310, 202311, 202312, 202401, 202402, 202403]
month_num =  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

isc_target_df = pd.DataFrame(columns=temp.columns)
isc_target_df.loc[0, 'tier_new'] = 'Independent Sales Company'
isc_target_df.loc[0, 'tier'] = 'Independent Sales Company'
isc_target_df.loc[0, 'pure tm'] = 'no'
isc_target_df.loc[0, 'tm_check'] = 'no'
isc_target_df.loc[0, 'target_order_intake_amount_eur'] = target_isc_should_be_by_month
isc_target_df.loc[0, 'order_intake_amount_eur'] = 0
isc_target_df = pd.concat([isc_target_df] * 12, ignore_index=True)
isc_target_df['year_month'] = fy_23
isc_target_df['m_num'] = month_num
isc_target_df['FY'] = temp['FY'][temp['FY'].dt.year == 2023].unique()

ibp_target_df = pd.DataFrame(columns=temp.columns)
ibp_target_df.loc[0, 'tier_new'] = 'Indirect Business Partner'
ibp_target_df.loc[0, 'tier'] = 'Indirect Business Partner'
ibp_target_df.loc[0, 'pure tm'] = 'no'
ibp_target_df.loc[0, 'tm_check'] = 'no'
ibp_target_df.loc[0, 'target_order_intake_amount_eur'] = target_ibp_should_be_by_month
ibp_target_df.loc[0, 'order_intake_amount_eur'] = 0
ibp_target_df = pd.concat([ibp_target_df] * 12, ignore_index=True)
ibp_target_df['year_month'] = fy_23
ibp_target_df['m_num'] = month_num
ibp_target_df['FY'] = temp['FY'][temp['FY'].dt.year == 2023].unique()

In [176]:
# additional country data addition

rs_df = pd.read_excel('data_files/rs_data.xlsx')
rs_df['pure tm'] = 'no'
rs_df['tier_new'] = rs_df['tier']
rs_df['tm_check'] = 'no'
rs_df['m_num'] = 1
rs_df['target_order_intake_amount_eur'] = 0

rs_df = rs_df[rs_df['indirect_direct'] != 'Direct']

rs_df = rs_df[list(set(temp.columns).intersection(set(rs_df.columns)))]

In [177]:
temp = pd.concat([temp, rs_df])
temp = pd.concat([temp, isc_target_df])
temp = pd.concat([temp, ibp_target_df])
temp = temp.reset_index(drop=True)

condition_tm = (temp['tm_check'] == 'no') & (temp['pure tm'] == 'no')

temp = temp[condition_tm]

temp = temp[['sold_to_customer', 'customer_name', 'sold_to_customer_n',
       'ec_eu_customer_n', 'company_code_n', 'countries', 'indirect_direct', 'tier',
       'tier_new', 'type', 'year_month', 'FY', 'bu2', 'for_bu',
       'sales_order_so', 'sales_person_n', 'order_intake_amount_eur',
        'target_order_intake_amount_eur', 'm_num']]

# Create a pivot table from the DataFrame
temp_b = temp.copy()
temp_b['FY'] = temp_b['FY'].dt.year
pivot_table1 = temp_b.pivot_table(index='tier_new', columns='FY', values='order_intake_amount_eur', aggfunc='sum')


# Create a dictionary mapping sold_to_customer values to corresponding countries
df_customers['sold_to_customer'] = df_customers['sold_to_customer'].astype(str)
customer_country_map = df_customers.set_index('sold_to_customer')['countries'].to_dict()

# Fill missing values in temp_df['countries'] using the mapping dictionary
temp['countries'] = temp['countries'].fillna(temp['sold_to_customer'].map(customer_country_map))

In [179]:
writer = pd.ExcelWriter(f'data_files/outcome/results_{year_month}.xlsx')
# Save each DataFrame to a separate sheet in the same file
temp.to_excel(writer, sheet_name='temp', index=False)
pivot_table1.to_excel(writer, sheet_name='pivot')
# Save the file
writer.close()

In [ ]:
# write previous pivot
writer = pd.ExcelWriter(f'data_files/outcome/last_pivot_{year_month}.xlsx')
pivot_table1.to_excel(writer, sheet_name='pivot')
writer.close()

In [78]:
# understanding difference between last data and new
last_pivot = pd.read_excel(f'data_files/outcome/last_pivot_{year_month}.xlsx')

last_pivot = last_pivot.set_index('tier_new')
last_pivot= last_pivot.round(2)

pivot_table = pivot_table1.reset_index()
pivot_table = pivot_table.set_index('tier_new')
pivot_table = pivot_table.round(2)

pivot_table - last_pivot